## Import Libraries

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Get the current working directory
pwd= os.getcwd()

In [3]:
df1 = pd.read_csv(pwd + "/202306121658.csv")
df_countrytiers = pd.read_excel(pwd + "/Tier1_List.xlsx")
df_managerslist = pd.read_excel(pwd + "/ManagersList.xlsx")
df_campmanagerslist = pd.read_excel(pwd + "/CampManagersList.xlsx")

In [4]:
index = df1[ (df1['COST'] == 0) & (df1['REVENUE'] ==0) ].index
df1.drop(index , inplace=True)

In [5]:
import re
words = ['suite', 'workspace','ads','advertis','video']
regex = '|'.join(map(re.escape, words))
# 'health|therapist|sales|private'
# keywords- lms, 3pl, edi,headless, knowledge/content, endpoint, server,project, health
# new keywords - soc 2, it , headless , discovery, billing, staffing, inventory
# dormant - laboratory, Supply Chain, corporate, therapist, event management, identity, power bi, medical staffing, ,forensic, ehr, enterprise
#lawyer,marketing,order, file, therap, analytics, document, cyber, workflow, ai, consulting

#removed 23aug - email,private
#removed 21Sept- mental,document
#added -ads, workable
#'product','task', 'ats' , gcp

# option 1: get first match
df1['category_first'] = (df1['Campaign_Query']
 .str.extract(f'(?i)({regex})', expand=False)
 .fillna('Other')
 )

# option 2: get all matches
df1['category_all'] = (df1['Campaign_Query']
 .str.extractall(f'(?i)({regex})')
 [0].groupby(level=0).agg(lambda x: ','.join(set(x)))
 .reindex(df1.index, fill_value='Other')
 )

In [6]:
df_merge_2 = pd.merge(df1, df_campmanagerslist, on='Label', how='left')

df_merge_2['MTD'] = np.where(df_merge_2['DATE']!= '[]', True, False)

df_merge_2['WTD'] = np.where(df_merge_2['DATE']!= '[]', True, False)


# using dt accessor to extract day               
df_merge_2["DATE"]= pd.to_datetime(df_merge_2[ "DATE"])
df_merge_2['DayOfMonth']=df_merge_2[ "DATE"].dt.day

# using dt accessor to extract month                  
#df_merge_2["DATE"]= pd.to_datetime(df_merge_2[ "DATE"])
#df_merge_2['Month']=df_merge_2[ "DATE"].dt.month

# using dt accessor to extract year                     
#df_merge_2["DATE"]= pd.to_datetime(df_merge_2[ "DATE"])
#df_merge_2['Year']=df_merge_2[ "DATE"].dt.year

df_merge_2["DATE"]= pd.to_datetime(df_merge_2[ "DATE"])
df_merge_2['Day_name']=df_merge_2[ "DATE"].dt.day_name()




In [7]:
#max(df_merge_2['DayOfMonth'])

#### Spend Checklist

In [8]:
df_merge_2.to_excel("Target vs Achievement_June_05-June_11.xlsx", index=False)



# I think the easiest way is using python-dateutil like this:

from datetime import date
from dateutil.relativedelta import relativedelta, MO

today = date.today()
last_monday = today + relativedelta(weekday=MO(-1))
print (last_monday)

In [9]:
#max(df_merge_2['Day_name'])

In [10]:
#df_merge_2['WTD'] = np.where(df_merge_2['DATE']>= last_monday, True, False)

In [11]:
df_merge_2.dtypes

DATE                   object
Site                   object
Label                  object
Campaign_Name          object
Campaign_Query         object
COST                  float64
REVENUE               float64
PROFIT                float64
Margin                float64
AdWords_Clicks          int64
AdSense_Clicks          int64
AdWords_CPC           float64
AdSense_CPC           float64
Conversion            float64
Demand_IMPRESSIONS      int64
Supply_IMPR             int64
category_first         object
category_all           object
Team Member            object
Manager                object
dtype: object